# Supermarket Ordering, Invoicing, and Sales

Joel Day, Nicholas Lee, and Christine Vu

Shiley-Marcos School of Engineering, University of San Diego

ADS 507: Practical Data Engineering

Professor Jonathan Sixt

February 8, 2023

***

## Data Description

### Invoices.csv

| Variable | Description  |
| --- | --- |
| Order Id | The order identification number |
| Date | The date the order was placed |
| Meal Id | The meal identification number |
| Company Id | The company identification number |
| Date of Meal | The date the meal was served |
| Participants | The number of people who participated in the meal |
| Meal Price | The cost of the meal |
| Type of Meal | The type of meal that was ordered |

### OrderLeads.csv

| Variable | Description  |
| --- | --- |
| Order Id | The order identification number |
| Company Id | The company identification number |
| Company Name | The name of the company associated with the order |
| Date | The date the order was placed |
| Order Value | The total value of the order |
| Converted | Whether or not the order was converted into a sale |

### SalesTeam.csv

| Variable | Description  |
| --- | --- |
| Sales Rep | The name of the sales representative |
| Sales Rep Id | The sales representative identification number |
| Company Name | The name of the company associated with the order |
| Company Id | The company identification number |

***

## Data Importing and Pre-processing

In [ ]:
# Packages
import datetime as dt
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re

import pymysql
from sqlalchemy import create_engine
import requests
import io
import os

import warnings
warnings.filterwarnings("ignore")

Import in CSV files

In [ ]:
# Function to Pull Raw CSV from GitHub and Convert to Pandas Dataframe Object

def github_to_pandas(raw_git_url):
    # Pull Raw CSV File from GitHub
    file_name = str(raw_git_url)
    pull_file = requests.get(file_name).content

    # Convert Raw CSV to Pandas Dataframe
    csv_df = pd.read_csv(io.StringIO(pull_file.decode('utf-8')))

    return csv_df

In [ ]:
# Pull CSV files from GitHub and Convert to Pandas Dataframe
invoice_df = github_to_pandas(
    "https://raw.githubusercontent.com/nlee98/ADS-507-Data-Engineering/main/Invoices.csv")

orderleads_df = github_to_pandas(
    "https://raw.githubusercontent.com/nlee98/ADS-507-Data-Engineering/main/OrderLeads.csv")

salesteam_df = github_to_pandas(
    "https://raw.githubusercontent.com/nlee98/ADS-507-Data-Engineering/main/SalesTeam.csv")

### Data Pre-processing

In [ ]:
# Find missing values
print("- Invoice Missing Values:\n", invoice_df.isnull().sum())
print("\n- Order Leads Missing Values:\n", orderleads_df.isnull().sum())
print("\n- Sales Team Missing Values:\n", salesteam_df.isnull().sum())

In [ ]:
# Data types of all columns
print("- Invoice Data Types:\n", invoice_df.dtypes)
print("\n- Order Leads Data Types:\n", orderleads_df.dtypes)
print("\n- Sales Team Data Types:\n", salesteam_df.dtypes)

In [ ]:
# Duplicated data
print("- Invoice Duplicated Values:", invoice_df.duplicated().sum())
print("- Order Leads Duplicated Values:", orderleads_df.duplicated().sum())
print("- Sales Team Duplicated Values:", salesteam_df.duplicated().sum())

***

## Explore CSV Files

### Invoice CSV

In [ ]:
invoice_df.head(3)

#### Transformations
* Add Underscores to each column name
* Transform Date and Date of Meal to date/datetime data types
* Time of day column
* Number of participants column

In [ ]:
# Replace spaces with underscores in all dataframe column names
invoice_df.columns = invoice_df.columns.str.replace(" ", "_")
orderleads_df.columns = orderleads_df.columns.str.replace(" ", "_")
salesteam_df.columns = salesteam_df.columns.str.replace(" ", "_")

In [ ]:
# Date to Date ("d-m-Y")
invoice_df["Date"] = pd.to_datetime(
    invoice_df["Date"], format='%d-%m-%Y')

In [ ]:
# Drop "+HH:MM:SS" to make all uniform to UTC timezone
invoice_df["Date_of_Meal"] = invoice_df["Date_of_Meal"].apply(
    lambda x: x.split("+")[0]
)

# Convert Date_of_Meal to Datetime format
invoice_df["Date_of_Meal"] = pd.to_datetime(
    invoice_df["Date_of_Meal"],
    format = "%Y-%m-%d %H:%M:%S"
)

In [ ]:
# Convert Date_of_Meal to Datetime format
invoice_df["Date_of_Meal"] = pd.to_datetime(
    invoice_df["Date_of_Meal"],
    format = "%Y-%m-%d %H:%M:%S"
)

In [ ]:
# Function defining hour of the day with the time of day
def time_of_day(x):
    day_hour = x.hour
    if (day_hour >= 5) and (day_hour <= 8): # 5am - 8am
        return "Early Morning"
    elif (day_hour > 8) and (day_hour <= 12): # 9am - 12pm
        return "Late Morning"
    elif (day_hour > 12) and (day_hour <= 15): # 1pm - 3pm
        return "Early Afternoon"
    elif (day_hour > 15) and (day_hour <= 19): # 4pm - 7pm
        return "Evening"
    elif (day_hour > 19) and (day_hour <= 23): # 8pm - 11pm
        return "Night"
    else: # 12am - 4am
        return "Late Night"

In [ ]:
# Apply time_of_day function to Date_of_Meal column
invoice_df["Part_of_Day"] = invoice_df["Date_of_Meal"].apply(time_of_day)

In [ ]:
# Add a field to count the number of participants
invoice_df['Number_of_Participants'] = invoice_df['Participants'].apply(lambda x: x.count("'")/2)
invoice_df['Number_of_Participants'] = invoice_df['Number_of_Participants'].astype(int)

invoice_df.head(5)

### Customer-Order Table
Connect the customer id to each order id the customer placed. This table will link the customer information to the invoice information.

In [ ]:
# Find all the occurrences of customer names then explode to convert values in lists to rows
cust = invoice_df['Participants'].str.findall(r"'(.*?)'").explode()

# Join with order id 
cust_order_df = invoice_df[['Order_Id']].join(cust)

# Factorize to encode the unique values in participants
cust_order_df['Customer_Id'] = cust_order_df['Participants'].factorize()[0] + 1
cust_order_df["Customer_Id"] = cust_order_df["Customer_Id"].astype(str)

# Rename Participants Column

cust_order_df.columns = ["Order_Id", "Participant_Name", "Customer_Id"]

# Add Last Updated Date
cust_order_df["Last_Updated"] = dt.date.today()

cust_order_df.head(9)

### Order Leads CSV
* Converted Column - Whether or not a order was converted into a sale

In [ ]:
orderleads_df.head(3)

In [ ]:
orderleads_df["Date"] = pd.to_datetime(orderleads_df["Date"])

### Sales Team CSV

In [ ]:
salesteam_df.head(3)

***

## Connection to MySQL Server

In [ ]:
# Manually Login to MySQL
mysql_username = str(input("Enter MySQL Username: "))
mysql_password = str(input("Enter MySQL Password: "))

mysql_conn = pymysql.connect(
    host = "localhost",
    port = int(3306),
    user = mysql_username,
    passwd = mysql_password
)

### Create Supermarket Database
* Tries to drop the database, if it previously existed
    - Otherwise, creates the database

In [ ]:
# Create ADS-507_Supermarket MySQL Database
## Drop the database to create an updated version if it exists
try :
    mysql_conn.cursor().execute(
        """
        DROP DATABASE ADS_507_Supermarket;
        """
    )
    mysql_conn.cursor().execute(
        """
        CREATE DATABASE IF NOT EXISTS ADS_507_Supermarket;
        """
    )
# Create the database if it has not done so before
except: 
    mysql_conn.cursor().execute(
        """
        CREATE DATABASE IF NOT EXISTS ADS_507_Supermarket;
        """
    )

# Navigate to Supermarket Database
mysql_conn.select_db("ADS_507_Supermarket")

print("ADS-507 Supermarket Database Created")

In [ ]:
mysql_conn.cursor().execute('USE ADS_507_Supermarket')

In [ ]:
mysql_conn.cursor().execute('SHOW TABLES')

In [ ]:
# Partition by the year for scalability
create_invoice_table = """
CREATE TABLE IF NOT EXISTS invoice (
    Order_Id VARCHAR(100) NOT NULL,
    Date DATE NOT NULL,
    Meal_Id VARCHAR(100) NOT NULL, 
    Company_Id VARCHAR(100) NOT NULL,
    Date_of_Meal DATETIME NOT NULL,
    Participants VARCHAR(255),
    Meal_Price SMALLINT,
    Type_of_Meal ENUM('Breakfast', 'Lunch', 'Dinner'),
    Part_of_Day ENUM('Early Morning', 'Late Morning', 'Early Afternoon', 'Night', 'Late Night'),
    Number_of_Participants TINYINT,
    INDEX (Date),
    INDEX (Order_Id)
)
PARTITION BY RANGE (YEAR(Date)) (
    PARTITION invoice_2014 VALUES LESS THAN (2015),
    PARTITION invoice_2015 VALUES LESS THAN (2016),
    PARTITION invoice_2016 VALUES LESS THAN (2017),
    PARTITION invoice_2017 VALUES LESS THAN (2018),
    PARTITION invoice_2018 VALUES LESS THAN (MAXVALUE)
)
;
"""

mysql_conn.cursor().execute(create_invoice_table);
print("Invoice Table Created")

In [ ]:
create_order_table = """
    CREATE TABLE IF NOT EXISTS orders(
        Order_Id VARCHAR(100) NOT NULL,
        Company_Id VARCHAR(100) NOT NULL,
        Company_Name VARCHAR(255),
        Date DATE,
        Order_Value SMALLINT,
        Converted TINYINT,
        PRIMARY KEY (Order_Id, Company_Id),
        INDEX (Company_Id),
        INDEX (Order_Id)
    )
;
"""

mysql_conn.cursor().execute(create_order_table);
print("Orders Table Created")

In [ ]:
# Foreign key added on company_id to link the salesteam to the orders table
create_salesteam_table = """
    CREATE TABLE IF NOT EXISTS salesteam(
        Sales_Rep VARCHAR(255),
        Sales_Rep_Id VARCHAR(100),
        Company_Name VARCHAR(255),
        Company_Id VARCHAR(100),
        FOREIGN KEY (Company_Id) REFERENCES orders(Company_Id),
        INDEX (Sales_Rep),
        INDEX (Company_Name)
    )
"""

mysql_conn.cursor().execute(create_salesteam_table);
print("Sales Table Created")

In [ ]:
create_customerorder_table = """
    CREATE TABLE IF NOT EXISTS customer_order(
        Order_Id VARCHAR(100),
        Participant_Name VARCHAR(255),
        Customer_Id VARCHAR(255),
        Last_Updated DATE,
        FOREIGN KEY (Order_Id) REFERENCES orders(Order_Id),
        INDEX (Participant_Name)
    )
"""

mysql_conn.cursor().execute(create_customerorder_table);
print("Customer_Order Table Created")

In [ ]:
# Create Engine to write to SQL table
engine = create_engine(
    "mysql+pymysql://{user}:{pw}@{host}/{db}".format(
        host = "localhost", 
        db = "ADS_507_Supermarket", 
        user = mysql_username, 
        pw = mysql_password))

## Load Dataframes as Tables into MySQL
* Invoice
* Orders
* Sales Lead
* Customer (cust_order_df)

In [ ]:
# Load invoice datafraome to SQL table
for i, df_row in invoice_df.iterrows():
    row_value = """
    INSERT INTO ADS_507_Supermarket.invoice VALUES (
        %s, %s, %s, %s, %s,
        %s, %s, %s, %s, %s)
        """
    mysql_conn.cursor().execute(row_value, tuple(df_row))

print("Successfully added data to invoice table")

In [ ]:
# Load orders datafraome to SQL table
for i, df_row in orderleads_df.iterrows():
    row_value = """
    INSERT INTO ADS_507_Supermarket.orders VALUES (
        %s, %s, %s, %s, %s, %s)
        """
    mysql_conn.cursor().execute(row_value, tuple(df_row))

print("Successfully added data to orders table")

In [ ]:
# Load salesteam datafraome to SQL table
for i, df_row in salesteam_df.iterrows():
    row_value = """
    INSERT INTO ADS_507_Supermarket.salesteam VALUES (
        %s, %s, %s, %s)
        """
    mysql_conn.cursor().execute(row_value, tuple(df_row))

print("Successfully added data to salesteam table")

In [ ]:
# Load customer_order datafraome to SQL table
for i, df_row in cust_order_df.iterrows():
    row_value = """
    INSERT INTO ADS_507_Supermarket.customer_order VALUES (
        %s, %s, %s, %s)
        """
    mysql_conn.cursor().execute(row_value, tuple(df_row))

print("Successfully added data to customer_order table")

### MySQL Transformations
* Views
    - Customer Stats - total number of orders by each customer, total amount each customer spent, and the average amount each spent
    - Company Metrics - For each company, the total amount and average amount of each invoice monthly are shown for each meal (and displaying their respective meal). In addition, the year-to-date amount collected and yearly total are presented.
    - Sales Rep Performance - Sales_Rep, Sales_Rep_Id, Company_Name, Company_Id, Profit_by_Sales_Rep
    - Customer Purchases - Customer_Name, Part_of_Day, Company_Name, Number_of_Purchases, Total_Spent
    - Converted Count: Total amount of converted sales by company
    - Average Meal Price: Average meal price by type of meal
    - Total Sales: Total sales by type of meal price for each year
    - Highest Average Meal Price: Company and meal type with the highest average meal price
    - Highest Breakfast: Companies with the highest average breakfast price
    - Average Participants: Average amount of participants for each type of meal
    - Difference Days: Difference in days between the date of meal and date the order was placed
    - Profit: Difference between the order value and meal price
    - Sales Rep More Than 500: Find the sales representatives who have not sold a meal that costs more than $500


In [ ]:
customer_stats = """
    CREATE VIEW customer_stats
        (customer_name, number_of_order, total_spent, average_spent)
        AS
        SELECT co.Participant_Name , COUNT(*), SUM(i.Meal_Price), AVG(i.Meal_Price)
        FROM customer_order AS co
            INNER JOIN invoice AS i
                ON co.Order_Id = i.Order_Id
        GROUP BY co.Participant_Name;
"""

mysql_conn.cursor().execute(customer_stats)
print("customer_stats view created")

In [ ]:
pd.read_sql("SELECT * FROM customer_stats LIMIT 3;", mysql_conn)

In [ ]:
company_metrics = """
    CREATE VIEW company_metrics
        (company_name, year, month, 
        meal_typ_sale, monthly_total,
        monthly_average, number_of_sales, year_to_date, yearly_total)
        AS
        SELECT o.Company_Name, YEAR(i.Date) AS Year, MONTHNAME(i.Date) AS Month,
            i.Type_of_Meal,
            SUM(i.Meal_Price) AS Monthly_Total,
            AVG(i.Meal_Price) AS Monthly_Average,
            COUNT(*) AS Number_of_Sales,
            SUM(SUM(i.Meal_Price))
                OVER (PARTITION BY o.Company_Name, Year(i.Date)
                        ORDER BY MONTH(STR_TO_DATE(month, '%M'))
                        ROWS UNBOUNDED PRECEDING) AS Year_to_Date,
            SUM(i.Meal_Price)
                OVER (PARTITION BY o.Company_Name, YEAR(i.Date)) AS Yearly_Total
        FROM orders AS o
            INNER JOIN invoice AS i
                ON o.Order_Id = i.Order_Id
        GROUP BY 1, 2, 3
        ORDER BY 1, 2, MONTH(STR_TO_DATE(month, '%M'))
"""

mysql_conn.cursor().execute(company_metrics)
print("company_metrics view created")

In [ ]:
pd.read_sql("SELECT * FROM company_metrics LIMIT 3;", mysql_conn)

In [ ]:
sales_rep_performance = """
    CREATE VIEW sales_rep_performance
        (Sales_Rep, Sales_Rep_Id, Company_Name,
        Company_Id, Profit_by_Sales_Rep)
        AS
        SELECT s.Sales_Rep AS Sales_Rep, 
            s.Sales_Rep_Id AS Sales_Rep_Id,
            s.Company_Name AS Company_Name,
            s.Company_Id AS Company_Id,
            SUM(i.Meal_Price) AS Profit_by_Sales_Rep
        FROM salesteam AS s
            INNER JOIN orders AS o
                ON s.Company_Id = o.Company_Id
            INNER JOIN invoice AS i
                ON o.Order_Id = i.Order_Id
        GROUP BY s.Sales_Rep, s.Company_Name
"""

mysql_conn.cursor().execute(sales_rep_performance)
print("sales_rep_performance view created")

In [ ]:
pd.read_sql("SELECT * FROM sales_rep_performance LIMIT 3;", mysql_conn)

In [ ]:
customer_purchases = """
    CREATE VIEW customer_purchases
        (Customer_Name, Part_of_Day, Company_Name,
        Number_of_Purchases, Total_Spent)
        AS
        SELECT c.Participant_Name, i.Part_of_Day,
            s.Company_Name, COUNT(i.Order_Id) AS Num_Purchases,
            SUM(i.Meal_Price) AS Meal_Price
        FROM invoice AS i
            INNER JOIN customer_order AS c
                ON i.Order_Id = c.Order_Id
            INNER JOIN salesteam AS s
                ON s.Company_Id = i.Company_Id
        GROUP BY 1,2
"""

mysql_conn.cursor().execute(customer_purchases)
print("customer_purchases view created")

In [ ]:
pd.read_sql("SELECT * FROM customer_purchases LIMIT 3;", mysql_conn)

In [ ]:
test_query = """
    SELECT *
    FROM company_metrics
    LIMIT 5;
"""

pd.read_sql(test_query, mysql_conn)

In [ ]:
test_query = """
    show tables;
"""

pd.read_sql(test_query, mysql_conn)

In [ ]:
sales_by_year = """
    CREATE VIEW sales_by_year
        AS
    SELECT DISTINCT EXTRACT(YEAR FROM Date) Year, COUNT(*) Total_Invoices 
    FROM invoice 
    GROUP BY Year;
"""
mysql_conn.cursor().execute(sales_by_year)
#pd.read_sql(sales_by_year, mysql_conn)

In [ ]:
test_query = """
    SELECT *
    FROM sales_by_year
    LIMIT 5;
"""

pd.read_sql(test_query, mysql_conn)

In [ ]:
total_percent_converted = """
    CREATE VIEW total_percent_converted
        AS
    SELECT Converted, Count(*)  
    FROM orders 
    GROUP BY Converted;
"""
mysql_conn.cursor().execute(total_percent_converted)
#pd.read_sql(sales_by_year, mysql_conn)

In [ ]:
test_query = """
    SELECT *
    FROM total_percent_converted
    LIMIT 5;
"""

pd.read_sql(test_query, mysql_conn)

In [ ]:
percent_converted = """
    CREATE VIEW percent_converted
        AS
    SELECT Company_Name, ROUND(((SUM(Converted)/Count(*))*100), 1) Percent_Converted 
    FROM orders 
    GROUP BY Company_Name;
"""
mysql_conn.cursor().execute(percent_converted)
#pd.read_sql(sales_by_year, mysql_conn)

In [ ]:
test_query = """
    SELECT *
    FROM percent_converted
    ORDER BY Percent_Converted DESC;
"""

pd.read_sql(test_query, mysql_conn)

In [ ]:
# converted_count: Total amount of converted sales by company

converted_count = """CREATE VIEW converted_count AS
                        SELECT Company_Name, SUM(Converted) AS Converted_Total
                        FROM orders 
                        GROUP BY Company_Name;"""
                        
mysql_conn.cursor().execute(converted_count)

test_query = """SELECT * FROM converted_count;"""

pd.read_sql(test_query, mysql_conn)

In [ ]:
# avg_meal_price: Average meal price

avg_meal_price = """CREATE VIEW avg_meal_price AS
                    SELECT Type_of_Meal, AVG(Meal_Price) as Average_Meal_Price
                    FROM invoice
                    GROUP BY Type_of_Meal;"""
                        
mysql_conn.cursor().execute(avg_meal_price)

test_query = """SELECT * FROM avg_meal_price
                ORDER BY Average_Meal_Price DESC;"""

pd.read_sql(test_query, mysql_conn)

In [ ]:
# total_sales: Total sales by type of meal price by year

total_sales = """CREATE VIEW total_sales AS
                    SELECT EXTRACT(YEAR FROM Date) AS Year, Type_of_Meal, SUM(Meal_Price) as Total_Sales 
                    FROM invoice
                    GROUP BY Year, Type_of_Meal 
                    ORDER BY Year ASC, Total_Sales DESC;"""
                        
mysql_conn.cursor().execute(total_sales)

test_query = """SELECT * FROM total_sales;"""

pd.read_sql(test_query, mysql_conn)

In [ ]:
# highest_avg_meal_price: Company and meal type with the highest average meal price

highest_avg_meal_price = """CREATE VIEW highest_avg_meal_price AS
                            SELECT Type_of_Meal, Company_Id, MAX(JSON_LENGTH(Participants, 1)) as Maximum_Participants  
                            FROM invoice
                            GROUP BY Type_of_Meal, Company_Id;"""
                        
mysql_conn.cursor().execute(highest_avg_meal_price)

test_query = """SELECT * FROM highest_avg_meal_price;"""

pd.read_sql(test_query, mysql_conn)

In [ ]:
# highest_breakfast: Companies with the highest average breakfast price

highest_breakfast = """CREATE VIEW highest_breakfast AS
                        SELECT Company_Id, AVG(Meal_Price) as Average_Breakfast_Price
                        FROM invoice
                        WHERE Type_of_Meal = 'Breakfast' 
                        GROUP BY Company_Id
                        ORDER BY Average_Breakfast_Price DESC;"""
                        
mysql_conn.cursor().execute(highest_breakfast)

test_query = """SELECT * FROM highest_breakfast;"""

pd.read_sql(test_query, mysql_conn)

In [ ]:
# avg_participants: Average amount of participants for each type of meal

avg_participants = """CREATE VIEW avg_participants AS
                        SELECT Type_of_Meal, AVG(JSON_LENGTH(Participants)) as Average_Participants_Per_Meal
                        FROM invoice
                        GROUP BY Type_of_Meal"""
                        
mysql_conn.cursor().execute(avg_participants)

test_query = """SELECT * FROM avg_participants;"""

pd.read_sql(test_query, mysql_conn)

In [ ]:
# difference_days: Difference in days between Date_of_Meal and Date

difference_days = """CREATE VIEW difference_days AS
                        SELECT Date, Date_of_Meal, DATEDIFF(Date_of_Meal, Date) AS Days_Between
                        FROM invoice;"""
                        
mysql_conn.cursor().execute(difference_days)

test_query = """SELECT * FROM difference_days;"""

pd.read_sql(test_query, mysql_conn)

In [315]:
# salesrep_morethan10: Find the sales representatives who have not sold a meal that costs more than $10

salesrep_morethan10 = """CREATE VIEW salesrep_morethan10 AS
                            SELECT DISTINCT st.Sales_Rep
                            FROM salesteam st
                            WHERE NOT EXISTS (
                                SELECT 1 FROM invoice i
                                WHERE i.Meal_Price > 100 AND i.Company_Id = st.Company_Id);"""
                        
mysql_conn.cursor().execute(salesrep_morethan10)

test_query = """SELECT * FROM salesrep_morethan10;"""

pd.read_sql(test_query, mysql_conn)

OperationalError: (1050, "Table 'salesrep_morethan10' already exists")

In [321]:
# salesrep_morethan500: Find the sales representatives who have not sold a meal that costs more than $500

salesrep_morethan500 = """CREATE VIEW salesrep_morethan500 AS
                            SELECT DISTINCT st.Sales_Rep
                            FROM salesteam st
                            WHERE NOT EXISTS (
                                SELECT 1 FROM invoice i
                                WHERE i.Meal_Price > 500 AND i.Company_Id = st.Company_Id);"""
                        
mysql_conn.cursor().execute(salesrep_morethan500)

test_query = """SELECT * FROM salesrep_morethan500;"""

pd.read_sql(test_query, mysql_conn)

,Sales_Rep
0,Lois Bowers
1,Latesha Tessitore
2,Billy Breece
3,Derek Walters
4,Alexandra Martino
...,...
71,Richard Swinford
72,Teresa Bilger
73,Dan Saez
74,Linda Carroll
